<h1 style="color:#00A6D6;">Introduction to Quantum Cryptography - Jupyter Notebooks</h1>
<h2 style="color:#00A6D6;">Chapter 10: Quantum Cryptography beyond Key 
Distribution </h2>
In this notebook, we will investigate several aspects of quantum cryptography beyond Key Distribution. We will look at a proposed protocol for bit commitment and take a look at its security using Uhlman's theorem from Chapter 4. We then continue to look at another protocol for coin flipping with some interesting properties!

* <a href="#uhl"> Cheating Bit Commitments using Uhlman's Theorem </a>

* <a href="#wcp"> Cheat Sensitive Weak Coin Flipping </a>

Notice that the line below will make Julia add and compile several packages. After compiling them it will print several info and warning messages.

In [ ]:
include("source/main.jl");

<a id="uhl"></a>
<h2 style="color:#00A6D6;">Cheating Bit Commitments using Uhlman's theorem </h2>

Let's recall Uhlman's theorem from Chapter 4, and investigate what it can tell us about cheating in a bit commitment protocol. Let's consider the following attempt at a simple bit commitment protocol:

<ul>
    <li>Commit phase:</li>
    <ul>
    <li>To commit $b = 0$, Alice prepares $$|\Phi\rangle = \frac{1}{\sqrt{2}}\left(|00\rangle + |11\rangle\right).$$ To commit $b=1$, she prepares $$|\Psi\rangle = \frac{1}{\sqrt{2}}\left(|+0\rangle + |-1\rangle\right).$$</li>
    <li>Alice sends the second qubit to Bob.</li>
    </ul>
    <li>Open phase:</li>
    <ul>
        <li>Alice sends $b$ to Bob, as well as the first qubit of the state above.</li>
        <li> Bob checks whether Alice measured in the correct basis: he measures in the basis $\{|\Phi\rangle\langle\Phi|,I-|\Phi\rangle\langle\Phi|\}$ for $b=0$, and in the basis $\{|\Psi\rangle\langle\Psi|,I-|\Psi\rangle\langle\Psi|\}$ for $b=1$.</li>
    </ul>
</ul>


In [ ]:
# Let's investigate the protocol

# Bit to be commmitted by Alice (if she is honest)
b = 0;

# Let's first define the states and measurements to be performed in the protocol
# States in the protocol
ketphi = [1 0 0 1]/sqrt(2);
Phi = ketphi'*ketphi;

ketpsi = [1 1 1 -1]/2;
Psi = ketpsi'*ketpsi;

# Measurements by Bob (to allow you to experiment more easily)
check0Good = Phi;
check0Cheat = I - Phi;
check1Good = Psi;
check1Cheat = I - Psi;

# Now the protocol
# Commit phase
# State chosen by Alice
if b == 0
    print("Commit: Alice honestly commits to b=0\n");
    rhoAB = Phi;
else
    print("Commmit: Alice honestly commits to b=1\n");
    rhoAB = Psi;
end
print("State prepared by Alice is: ",rhoAB,"\n");

# Open Phase
# Bit to be opened by Alice
openB = b; # Alice is honest

print("Open: Alice sends ", openB, " to Bob.\n");

# Now Bob measures
if openB == 0
    pgood = tr(check0Good * rhoAB);
else
    pgood = tr(check1Good * rhoAB);
end
# Sample from {0,1} where p1 is given 
rndDist = Binomial(1,1-pgood);
outcome = rand(rndDist);

if outcome == 0
    print("Open: Bob acccepts!\n");
else
    print("Open: Bob rejects: Alice is cheating!!\n");
end


<h3 style="color:#00A6D6;"> Exercise 1</h3>
Let's first investigate running the protocol. First execute the code above. What do you observe? What happens if Alice tries to open another bit, i.e. openB is not the same as b? You may wish to explore using other measurements or states.

In [ ]:
# Play with the code above, or copy paste it here to make more radical changes.

Let's now investigate whether Alice can cheat, recalling Uhlman's theorem. Recall that  
when two **rank-one** density matrices $\Phi_{AB}$ and $\Psi_{AB}$ have the same partial trace - $Tr_{A}(\Phi_{AB}) = Tr_A(\Psi_{AB})$ - it follows - in accordance with Uhlman's theorem - that the matrices are equivalent up to a local unitary on $A$. This means that if Bob has qubit $B$, then Alice can - by applying this precise unitary! - transform one state to the other. 

<h3 style="color:#00A6D6;"> Exercise 2</h3>

Are the two states $\Psi_{AB}$ and $\Phi_{AB}$ in our protocol equivalent up to a local unitary? Can you guess what that unitary is? Explain how Alice could use this fact to cheat. 

In [ ]:
# Complete the code below to investigate the exercise!

phiB = partialTrace(Phi,"B"); # returns the reduced state on A (traces out the B system)
display(phiB)

<a id="wcp"></a>
<h2 style="color:#00A6D6;">Cheat Sensitive Weak Coin Flipping</h2>
Let us now investigate a different coin flipping protocol than we saw in the book. Specifically, this protocol was discovered by <a href="https://arxiv.org/abs/quant-ph/0202118">Spekkens and Rudolph</a> and has a nice and peculiar quantum feature: it is cheat sensitive! That is, we will try and exploit the fact that quantum measurements can cause disturbance to try and detect if some of the parties in the protocol were trying to cheat.

The protocol is relatively simple, and can be instantiated for different choices of states $|\Psi\rangle_{AB}$ and measurements $\{E_0,E_1\}$. 
<ul>
    <li>Alice prepares a state $|\Psi\rangle_{AB}$ and sends the $B$ qubit to Bob.</li>
    <li>Bob measures using the POVM $\{E_0,E_1\}$. </li>
    <li> If Bob's outcome is 0, then
        <ul>
            <li>Bob sends qubit $B$ back to Alice. </li>
            <li>Alice measures both qubits using the measurement $\{M_0=|\Phi\rangle\langle \Phi|,M_1=I - |\Phi\rangle\langle \Phi|\}$, where $$|\Phi\rangle = I\otimes E_0|\Psi\rangle/\sqrt{\langle\Psi|I \otimes E_0|\Psi\rangle}.$$ </li>
            <li>If Alice's outcome is 0, then Bob wins. If Alice's outcome is 1, then Alice catches Bob cheating. </li>
        </ul></li>
    <li> If Bob's outcome is 1, then Alice sends her qubit $A$ to Bob.
     <ul>
            <li>Alice sends her qubit $A$ to Bob. </li>
            <li>Bob measures both qubits using the measurement $\{M_0=|\Phi\rangle\langle \Phi|,M_1=I - |\Phi\rangle\langle \Phi|\}$, where $$|\Phi\rangle = I\otimes E_1|\Psi\rangle/\sqrt{\langle\Psi|I \otimes E_1|\Psi\rangle}.$$ </li>
            <li>If Bob's outcome is 0, then Alice wins. If Bob's outcome is 1, then Bob catches Alice cheating. </li>
        </ul></li>
</ul>

Below we will examine the protocol for two types of state and measurements as suggested in their paper. We choose $$|\Psi\rangle = \sqrt{x}|00\rangle + \sqrt{(1-x)}|11\rangle\ , $$ for some choice of  $1/2 < x \leq 1$. In the first case, the measurements are given by $$E_0 = \frac{1}{2x}|0\rangle\langle0|, E_1 = I -E_0.$$

In [ ]:
# The following function will run the protocol once, and return to use the outcome - assuming both parties honestly follow the protocol. Inputs are
# 
# ketPsi: Psi as a ket vector
# actualPsi: actual Psi prepared by Alice if she is cheating (or experiences errors)
# E0: POVM, assumded to be diagonal
# 
function rsWCF(ketPsi, actualPsi, E0)

     # Result to be returns 0 Alice wins, 1 Bob wins, 2 Alice cheated, 3 Bob cheated
    answer = 0;
    
    # Compute E1
    id = [1 0; 0 1];
    E1 = id - E0;
    
    # To compute the post measurement state of Alice and Bob, this code assumes sqrt(E0) can be simply computed thus (ok for the examples given)
    sE0 = sqrt(kron(id,E0));
    sE1 = sqrt(kron(id,id-E0));
    
    # The state is given by Psi
    # Bob now measures his qubit using {E0,E1}. Due to numerical errors we compute both directly.
    p0 = tr(kron(id,E0) * actualPsi);
    p1 = tr(kron(id,E1) * actualPsi);

    # Sample from {0,1} where p1 is given 
    rndDist = Binomial(1,1-p0);
    outcome = rand(rndDist);

    if outcome == 0
        # Bob measured outcome given by E0 and sends qubit B back to Alice
        # print("Bob sends B back to Alice\n");
    
        # Compute the post measurement state of Alice and Bob
        rhoPost = (sE0*actualPsi*sE0')/p0;
   
        # Alice measures using the measurement determined by the post-measurement states
        Phi = (sE0*ketPsi'/sqrt(p0))';
        M0 = Phi' * Phi;
        q0 = tr(M0*rhoPost);
        
        # Sample from {0,1} where p1 is given, rounding errors can make q1 slightly larger than 1 
        rndDistCheck = Binomial(1,max(0,1-q0));
        outcomeAlice = rand(rndDistCheck);

        if outcomeAlice == 0
           # print("Bob wins! \n");
           answer = 1;
        else
           # print("Bob is cheating!! \n");
           answer = 3;
        end
    else
         # Bob measured outcome given by E0 and Alice sends qubit A to Bob
        
        # Compute the post measurement state of Alice and Bob, this code assumes sqrt(E0) can be simply computed thus (ok for the examples given)
        rhoPost = (sE1 * actualPsi * sE1')/p1;
     
        # Alice measures using the measurement determined by the post-measurement states
        Phi = (sE1*ketPsi'/sqrt(p1))';
        M0 = Phi' * Phi;
        q0 = tr(M0*rhoPost);
 
        # Sample from {0,1} where p1 is given. Rounding errors can make q1 slightly larger than 1 
        rndDistCheck = Binomial(1,max(0,1-q0));
        outcomeBob = rand(rndDistCheck);

        if outcomeBob == 0
            # print("Alice wins! \n");
            answer = 0;
        else
            # print("Alice is cheating!! \n");
            answer = 2;
        end
    end

    return answer
end
   

<h3 style="color:#00A6D6;"> Exercise 3</h3>
Let us first investigate the protocol as a given with the state $|\Psi\rangle$ and measurement operator $E_0$ specified above. Use the above function to help you run the protocol, where in the honest case the actual state used by Alice is of course identical to the one defining the measurements in the protocol. Run the protocol multiple times, how often does Alice win, and how often does Bob win?

In [ ]:

# run the protocol (Alice and Bob honest) for a specific value of x
function runWCPV1(x)
    # Define Psi
    ketPsi = sqrt(x)*  [1 0 0 0] + sqrt(1-x) *  [0 0 0 1];
    actualPsi = ketPsi'* ketPsi;

    # Defined E0
    v0 = [1 0];
    E0 = 1/(2 * x) * v0' * v0; 
    # run the protocol
    answer = rsWCF(ketPsi,actualPsi,E0);

    return answer
end

# Set 1/2 < x < 1
x = 0.51;

# Count how often each win or cheat
aliceWins = 0;
bobWins = 0;
aliceCheats = 0;
bobCheats = 0;

# Run the protocol many times
n = 1000;
for j = 1:n
        answer = runWCPV1(x);
        if answer == 0
            aliceWins = aliceWins + 1;
        elseif answer == 1
            bobWins = bobWins + 1;
        elseif answer == 2
            aliceCheats = aliceCheats + 1;
        elseif answer == 3
            bobCheats = bobCheats + 1;
    end
end

print("Estimated probability that Alice wins: ",aliceWins/n, "\n");
print("Estimated probability that Bob wins: ", bobWins/n,"\n");


<h3 style="color:#00A6D6;"> Exercise 4</h3>
Let's investigate what happens in the honest case for different values of $x$. 

In [ ]:
# Helper function to let us understand the behaviour of Alice winning. n is the number of times we will execute the protocol
function estimateAliceWinning(x,n) 
    aliceWins = 0;
    for j = 1:n
        answer = runWCPV1(x);
        if answer == 0
            aliceWins = aliceWins + 1;
        end
    end
    return (aliceWins/n);
end

# Your code here

<h3 style="color:#00A6D6;"> Exercise 5</h3>
Let us now investigate what happens if Alice does not quite prepare the state dictated by the protocol, but chooses a different input state.
<ul>
    <li>First, try a noisy version of the state - for example by applying a bit flip with very low probability. What do you observe?</li>
    <li>Can you think of states to try that may give Alice an edge in the protocol, while not getting caught by Bob too often?</li>
</ul>
You may also try all together other cheating strategies by copy pasting the protocol above, and adapting it to your needs.

In [ ]:
# Your code goes here

<h3 style="color:#00A6D6;"> Exercise 6</h3>
In their paper, Spekkens and Rudolph also suggest using the same state $|\Psi\rangle$ as above with a different measurement given by operator $$E_0 = \left(1- \frac{1}{2x}\right) |0\rangle\langle 0| + |1\rangle\langle 1| \ , $$
with $1/2 < x \leq 1$. 
How does that change the behavior of the protocol above? Or any cheating strategy you thought of? Or maybe the noise tolerance of the protocol?

In [ ]:
# Your code goes here